# Sentiment Analysis con KNN
## Clasificador en C++ 💪💪
Vamos a probar a nuestro bichito


Definir los path al ejecutable de python 3.6 y sus librerías,
de acuerdo al virtual env que estén corriendo.

In [10]:
!cd .. && git submodule init
!cd .. && git submodule update
!cd .. && mkdir build
!cd ../build/ && rm -rf *
!cd ../build && cmake \
  -DPYTHON_EXECUTABLE="$(which python3)" \
  -DCMAKE_BUILD_TYPE=Release ..
!cd ../build && make install

-- The C compiler identification is GNU 7.4.0
-- The CXX compiler identification is GNU 7.4.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
Release mode
-- Found PythonInterp: /usr/bin/python3 (found version "3.6.8") 
-- Found PythonLibs: /usr/lib/x86_64-linux-gnu/libpython3.6m.so
-- pybind11 v2.3.dev0
-- Performing Test HAS_FLTO
-- Performing Test HAS_FLTO - Success
-- LTO enabled
CMAKE_INSTALL_PREFIX=/home2/clinux01/Desktop/Metodos-TP2
-- Configuring done
-- Generating done
-- Build files have been written to: /home2/clinux01/Desktop/Meto

In [1]:
# Verifico la correcta instalación. Si no falla el import está OK
!pwd
!python3 --version
import sentiment


/home2/clinux01/Desktop/Metodos-TP2/notebooks
Python 3.6.8


In [2]:
%load_ext autoreload
%autoreload 2
import pandas as pd

!cd ../data && tar -xvf *.tgz

df = pd.read_csv("../data/imdb_small.csv", index_col=0)

print("Cantidad de documentos: {}".format(df.shape[0]))

tar: *.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now
Cantidad de documentos: 12500


In [3]:
df.describe()

,type,review,label,file
count,12500,12500,12500,12500
unique,2,12469,2,12085
top,test,There are many different versions of this one ...,neg,798_1.txt
freq,6275,2,6322,2


In [15]:
text_train = df[df.type == 'train']["review"]
label_train = df[df.type == 'train']["label"]
text_train = text_train
label_train = label_train

text_test = df[df.type == 'test']["review"]
label_test = df[df.type == 'test']["label"]
text_test = text_test[0:1000]
label_test = label_test[0:1000]

print("Cantidad de instancias de entrenamiento = {}".format(len(text_train)))
print("Cantidad de instancias de test = {}".format(len(text_test)))


Cantidad de instancias de entrenamiento = 6225
Cantidad de instancias de test = 1000


In [16]:
print("Class balance : {} pos {} neg".format(
    (label_train == 'pos').sum(),
    (label_train == 'neg').sum() ))


Class balance : 3081 pos 3144 neg


In [17]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_df=0.90, min_df=0.01, max_features=500)

vectorizer.fit(text_train)

X_train, y_train = vectorizer.transform(text_train), (label_train == 'pos').values
X_test, y_test = vectorizer.transform(text_test), (label_test == 'pos').values

In [18]:
#ESTO SUPUESTAMENTE ENTRENA PCA

#import sentiment
#from sentiment import PCA, KNNClassifier
#alpha = 50
#pca = PCA(alpha)
#pca.fit(X_train.toarray())
#X_train = pca.transform(X_train)
#X_test = pca.transform(X_test)

y_test2 = []
for i in range(len(y_test)):
    y_test2.insert(i,0)

for i in range(len(y_test)):
    if y_test[i] == True:
        y_test2[i] = 1
    else :
        y_test2[i] = 0
        
y_train2 = []
for i in range(len(y_train)):
    y_train2.insert(i,0)

for i in range(len(y_test)):
    if y_train[i] == True:
        y_train2[i] = 1
    else :
        y_train2[i] = 0



In [19]:
%%time

import sentiment
from sklearn.metrics import accuracy_score

#import sentiment
from sentiment import PCA, KNNClassifier

print("k,accuracy")
for k in range (100,101,5):
    clf = sentiment.KNNClassifier(k)
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    #print(y_pred)
    res = 0
    for i in range(len(y_test2)):
        if abs(y_test2[i] - y_pred[i])<1e-4:
            res =res + 1
    #acc = accuracy_score(y_test2, y_pred)
    print("{},{}".format(k,res/len(y_pred)))

print("k,alpha,accuracy")

for alpha in range(1,100,5): 
    pca = PCA(alpha)
    pca.fit(X_train.toarray())
    X_train2 = pca.transform(X_train)
    X_test2 = pca.transform(X_test)
    for k in range(1,100,5):

        clf = sentiment.KNNClassifier(k)
        clf.fit(X_train2,y_train)
        y_pred = clf.predict(X_test2)
        #print(y_pred)
        res = 0
        for i in range(len(y_test2)):
            if abs(y_test2[i] - y_pred[i])<1e-4:
                res =res + 1
        #acc = accuracy_score(y_test2, y_pred)
        print("{},{},{}".format(k,alpha,res/len(y_pred)))
    ''''''

k,accuracy
1,0.577
6,0.617
11,0.632
16,0.654
21,0.652
26,0.647
31,0.66
36,0.646
41,0.648
46,0.641
51,0.648
56,0.626
61,0.641
66,0.636
71,0.636
76,0.639
81,0.639
86,0.645
91,0.635
96,0.643
101,0.633
106,0.641
111,0.639
116,0.642
121,0.643
126,0.638
131,0.643
136,0.644
141,0.64
146,0.639
151,0.646
156,0.646
161,0.65
166,0.64
171,0.637
176,0.636
181,0.637
186,0.632
191,0.629
196,0.629
k,alpha,accuracy
1,1,0.502
6,1,0.492
11,1,0.512
16,1,0.509
21,1,0.521
26,1,0.527
31,1,0.493
36,1,0.479
41,1,0.475
46,1,0.499
51,1,0.497
56,1,0.493
61,1,0.492
66,1,0.482
71,1,0.485
76,1,0.483
81,1,0.486
86,1,0.482
91,1,0.489
96,1,0.493
1,6,0.541
6,6,0.521
11,6,0.522
16,6,0.555
21,6,0.554
26,6,0.576
31,6,0.566
36,6,0.571
41,6,0.576
46,6,0.56
51,6,0.575
56,6,0.582
61,6,0.575
66,6,0.574
71,6,0.575
76,6,0.568
81,6,0.568
86,6,0.568
91,6,0.568
96,6,0.575
1,11,0.518
6,11,0.532
11,11,0.556
16,11,0.576
21,11,0.573
26,11,0.569
31,11,0.593
36,11,0.588
41,11,0.586
46,11,0.59
51,11,0.602
56,11,0.597
61,11,0.601
66,11,0.6


In [20]:
%%time
from sklearn.metrics import accuracy_score
y_pred = clf.predict(X_test)
print(y_pred)


acc = accuracy_score(y_test, y_pred)
print("Accuracy: {}".format(acc))

[1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1.
 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1.
 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1.
 0. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1.
 1. 0. 0. 0. 0. 0. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 0.
 1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1.
 1. 1. 0. 0. 0. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 0. 1.
 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.
 1. 0. 1. 0. 1. 0. 1. 1. 1. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 1. 0. 1. 1. 1.
 1. 1. 0. 1. 1. 1. 0. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1.
 1. 0. 1. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 0. 0. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1.
 1. 0. 1. 1. 1. 0. 1. 1. 1. 0. 0. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0.
 1. 1. 1. 0. 1. 1. 0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 1.

In [19]:
import numpy as np

X_ahre = X_train[:4,:4]
y_ahre = y_train[:4]
fitness = np.ndarray(shape=(1,4), dtype=float, order='F')
print(fitness)
X_que = np.asarray(X_train)

np.ndarray(shape=(2,2), dtype=float, order='F')

print(X_ahre)
print(y_ahre)
print(fitness)
clf = sentiment.KNNClassifier(1)
clf.fit(X_train,fitness)
res=0
for i in range(4):
    if abs(y_ahre[i] - fitness[0][i]) < :
        res =res+1
print(res)

#Ignora todo lo anterior
#Si ejecutas clf.fit te dice el tipo de dato que espera.
#Espera que le pases dos numpy.ndarray de floats, pero declararlas es un
#quilombo. Esto es raro porque antes no haciamos nada y andaba. Asi que 
#fijate que si pasas el array de bools a otra cosa deberias usar 
#el tipo de dato ese que dije. Igual voy a consultar con otros.
#Lo bueno es que knn esta checkeado mas o menos :S

SyntaxError: invalid syntax (<ipython-input-19-35a2247f46db>, line 18)

array([[0.0e+000, 4.9e-324, 9.9e-324, 1.5e-323]])